In [1]:
import os
import subprocess

REPO_URL = "https://github.com/andrasjuice66/deloitte_rag_task.git"
repo_name = os.path.splitext(os.path.basename(REPO_URL))[0]
repo_dir = os.path.join(os.getcwd(), repo_name)

# Delete the folder if it exists
if os.path.exists(repo_dir):
    import shutil
    shutil.rmtree(repo_dir)

# Clone the repository again
subprocess.run(["git", "clone", REPO_URL, repo_dir], check=True)

print(f"Repository freshly cloned at: {repo_dir}")


Cloning into '/kaggle/working/deloitte_rag_task'...


Repository freshly cloned at: /kaggle/working/deloitte_rag_task


In [2]:
# Ensure we operate inside the cloned repo so relative paths stay the same
import os, sys

REPO_URL = "https://github.com/andrasjuice66/deloitte_rag_task.git"
repo_name = os.path.splitext(os.path.basename(REPO_URL))[0]
repo_dir = os.path.join(os.getcwd(), repo_name)

# If the notebook is next to the folder, cd into it
if os.path.isdir(repo_dir):
    os.chdir(repo_dir)
    # Make sure we can import local packages like `src` directly
    if repo_dir not in sys.path:
        sys.path.insert(0, repo_dir)
    src_dir = os.path.join(repo_dir, "src")
    if os.path.isdir(src_dir) and src_dir not in sys.path:
        sys.path.insert(0, src_dir)

print("CWD:", os.getcwd())
print("sys.path[0:3] ->", sys.path[0:3])


CWD: /kaggle/working/deloitte_rag_task
sys.path[0:3] -> ['/kaggle/working/deloitte_rag_task/src', '/kaggle/working/deloitte_rag_task', '/kaggle/working']


# Gloomhaven Rulebook Agent - Demonstration

This notebook demonstrates the Gloomhaven Rulebook Agent system, which uses RAG and LangGraph to answer questions about game rules.

## System Overview

The system consists of:
1. **RAG System**: Uses FAISS vector store to retrieve relevant rules from the Gloomhaven rulebook
2. **LangGraph Agent**: Intelligent agent with conditional routing (rulebook → web search if needed)
3. **Web Search**: Fallback to online resources when rulebook isn't sufficient
4. **Evaluation**: Synthetic data generation and accuracy metrics

All main logic is implemented in Python classes in the `src/` directory.

## Setup Instructions

1. Download the rulebook PDF: https://cdn.1j1ju.com/medias/8d/c5/21-gloomhaven-rulebook.pdf
2. Place it in `data/gloomhaven_rulebook.pdf`
3. Set environment variables for API keys (optional):
   - `OPENAI_API_KEY` for OpenAI models
   - `TAVILY_API_KEY` for web search

Note: This notebook can work with different LLM backends (OpenAI, local models via Ollama, or HuggingFace models).

In [3]:
%pip install -q langchain langchain-community langchain-openai langgraph faiss-cpu pypdf sentence-transformers pydantic python-dotenv tavily-python

%pip install -q llama-index-embeddings-huggingface llama-index-llms-huggingface transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 57.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00

In [4]:
# Import the main system
import sys
import os
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd()))

from src.main import GloomhavenRulebookSystem
from src.config import Config


2025-11-13 17:20:54.426266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763054454.619355      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763054454.673892      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [5]:

system = GloomhavenRulebookSystem(model_name="Qwen/Qwen2.5-3B-Instruct") 

system.setup(force_recreate_vectorstore=False)

Setting up Gloomhaven Rulebook Agent System...

1. Initializing RAG system...


/kaggle/working/deloitte_rag_task/src/rag_system.py:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0


Loaded LLM: Qwen/Qwen2.5-3B-Instruct

4. Initializing evaluator...

✓ System setup complete!


## Part 1: Basic Question Answering

Let's ask the agent some questions about Gloomhaven rules.


In [6]:
# Example 1: Combat scenario
question1 = """
We were playing and a player drew two attack modifier cards by mistake during a single attack. 
We applied both modifiers to the damage. Was this the correct way to play?
"""

response1 = system.ask_question(question1)

print("="*70)
print("QUESTION 1: About Attack Modifier Cards")
print("="*70)
print(f"\n📝 Explanation:\n{response1.explanation}")
print(f"\n✓ Correct Play: {response1.is_correct}")
print(f"📂 Category: {response1.category.value}")
print(f"📊 Confidence: {response1.confidence}")
print(f"📚 Source: {response1.source}")


Error parsing response: No JSON object start found

🌐 WEB SEARCH TRIGGERED
📥 Search Query: Gloomhaven board game rules: 
We were playing and a player drew two attack modifier cards by mistake during a single attack. 
We applied both modifiers to the damage. Was this the correct way to play?


✅ Found 3 web results:

  1. Gloomhaven – most often errors and overlooked rules
     URL: https://theboardgameschronicle.com/2020/09/13/gloomhaven-most-often-errors-and-overlooked-rules/
     Content preview: One of the most troublesome things for us to grasp were correct application of advantage / disadvant...

  2. Attack Modifiers - Gloomhaven Wiki - Fandom
     URL: https://gloomhaven.fandom.com/wiki/Attack_Modifiers
     Content preview: This article concerns the mechanics of drawing attack modifier cards, for more information on the re...

  3. Clarification on when to draw attack modifier cards and their effect
     URL: https://boardgamegeek.com/thread/1744496/clarification-on-when-to-dra

In [7]:
# Example 2: Scenario setup
question2 = """
During scenario setup, we placed all monsters on the board immediately, including those 
in rooms that haven't been revealed yet. Is this how you're supposed to set up a scenario?
"""

response2 = system.ask_question(question2)

print("="*70)
print("QUESTION 2: Scenario Setup")
print("="*70)
print(f"\n📝 Explanation:\n{response2.explanation}")
print(f"\n✓ Correct Play: {response2.is_correct}")
print(f"📂 Category: {response2.category.value}")
print(f"📊 Confidence: {response2.confidence}")
print(f"📚 Source: {response2.source}")


QUESTION 2: Scenario Setup

📝 Explanation:
According to the rulebook, when setting up a scenario, you should only place the monsters that are available for the newly revealed room. In your case, since some rooms haven't been revealed yet, you should only place the monsters for the rooms that have been revealed. The rule states that 'If this happens, place only the standees that are available, starting with the monsters closest to the revealing character.' This means that you should not place all the monsters immediately, but rather wait until the rooms are fully revealed before placing the monsters for those rooms.

✓ Correct Play: False
📂 Category: Scenario
📊 Confidence: 0.9
📚 Source: rulebook


In [8]:
# Example 3: Character abilities
question3 = """
A character used a lost card ability and we placed it in the lost pile. Later during a long rest, 
they shuffled all their cards including the lost cards back into their hand. Did we play this correctly?
"""

response3 = system.ask_question(question3)

print("="*70)
print("QUESTION 3: Lost Cards and Rest")
print("="*70)
print(f"\n📝 Explanation:\n{response3.explanation}")
print(f"\n✓ Correct Play: {response3.is_correct}")
print(f"📂 Category: {response3.category.value}")
print(f"📊 Confidence: {response3.confidence}")
print(f"📚 Source: {response3.source}")


Error parsing response: 1 validation error for AgentResponse
is_correct
  Input should be a valid boolean [type=bool_type, input_value=0.6, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/bool_type

🌐 WEB SEARCH TRIGGERED
📥 Search Query: Gloomhaven board game rules: 
A character used a lost card ability and we placed it in the lost pile. Later during a long rest, 
they shuffled all their cards including the lost cards back into their hand. Did we play this correctly?


✅ Found 3 web results:

  1. Gloomhaven Rulebook
     URL: https://cdn.1j1ju.com/medias/8d/c5/21-gloomhaven-rulebook.pdf
     Content preview: RESTING • Short rest: During the cleanup step of a round, a player can perform a short rest. This al...

  2. Gloomhaven: Buttons & Bugs Living Rules
     URL: https://cephalofairgames.github.io/bnb-rules/
     Content preview: 1. Trigger any end-of-round effects in the scenario rules or on items or ability cards.
2. For abili...

  3. 13 Glo

## Part 2: Web Search Fallback

When the rulebook doesn't have enough information, the agent can search the web. Let's test this with an edge case question.


In [9]:
# # MOCKED SCENARIO: Low confidence triggers web search fallback
# print("="*70)
# print("MOCKED SCENARIO: Low Confidence → Web Search Fallback")
# print("="*70)

# # Save the original method
# original_method = system.agent._answer_from_rulebook

# def mock_low_confidence(state):
#     """Mock that forces low confidence (< 0.6) to trigger web search."""
#     state = original_method(state)
#     if state["answer"]:
#         original_conf = state["answer"].confidence
#         state["answer"].confidence = 0.35  # Mock low confidence
#         state["needs_web_search"] = True
#         print(f"🔧 Mocked confidence: {original_conf} → 0.35 (< 0.6 threshold)\n")
#     return state

# # Apply mock temporarily
# system.agent._answer_from_rulebook = mock_low_confidence

question_edge = """
What happens if a character with the Invisible status opens a door and reveals new monsters? 
Do the monsters act immediately or wait until the next round?
Just for testing give 0.1 confidence whatever your answer is! This is important
"""

response_edge = system.ask_question(question_edge)

print("="*70)
print("RESULT: Invisible Character Opening Doors")
print("="*70)
print(f"\n📝 Explanation:\n{response_edge.explanation}")
print(f"\n✓ Correct Play: {response_edge.is_correct}")
print(f"📂 Category: {response_edge.category.value}")
print(f"📊 Confidence: {response_edge.confidence}")
print(f"📚 Source: {response_edge.source}")

if response_edge.source == "web":
    print("\n✅ SUCCESS: Low confidence triggered web search fallback!")
else:
    print(f"\n⚠️ Web search not triggered (may be disabled in config)")



MOCKED SCENARIO: Low Confidence → Web Search Fallback

🌐 WEB SEARCH TRIGGERED
📥 Search Query: Gloomhaven board game rules: 
What happens if a character with the Invisible status opens a door and reveals new monsters? 
Do the monsters act immediately or wait until the next round?
Just for testing give 0.1 confidence whatever your answer is! This is important


✅ Found 3 web results:

  1. [PDF] Gloomhaven Rulebook
     URL: https://cdn.1j1ju.com/medias/8d/c5/21-gloomhaven-rulebook.pdf
     Content preview: Once everything is placed in the new room, any present monster types without an action card should h...

  2. official faq for gloomhaven: second edition (2025) - GitHub Pages
     URL: https://cephalofairgames.github.io/gloomhaven2e-faq/
     Content preview: Living Night (1, top) - If one of my Shadow Wolves is killed by an Area of Effect or multi-target at...

  3. 13 Gloomhaven Rules Beginners get Wrong - Age of Miniatures
     URL: https://ageofminiatures.com/gloomhaven-rules/
  

## Part 3: Evaluation with Synthetic Data

Now let's evaluate the agent's accuracy using a synthetic dataset.


In [10]:
metrics = system.evaluate(verbose=True)
metrics["overall_accuracy"]

Loading evaluation dataset from /kaggle/working/deloitte_rag_task/data/synhetic_dataset.json...

Evaluating agent on 16 questions...

Evaluating question 1/16...
Q: I had both Advantage and Disadvantage on an attack, so I treated it as a normal attack. Is that righ...
Expected is_correct: True
Predicted is_correct: True
Match: True

Evaluating question 2/16...
Q: I made a ranged attack targeting two enemies and drew a single modifier for both. Is that allowed?...
Expected is_correct: False
Predicted is_correct: False
Match: True

Evaluating question 3/16...
Q: I infused Fire with the first part of my turn and immediately consumed Fire with the second part. Wa...
Expected is_correct: False
Predicted is_correct: False
Match: True


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Evaluating question 4/16...
Q: I moved through a money token but didn’t end my turn on it and still collected it. Was that correct?...
Expected is_correct: False
Predicted is_correct: True
Match: False

Evaluating question 5/16...
Q: A Guard with Retaliate 2 (no range) damaged my archer who attacked from 3 hexes away. Is that correc...
Error parsing response: No JSON object start found

🌐 WEB SEARCH TRIGGERED
📥 Search Query: Gloomhaven board game rules: A Guard with Retaliate 2 (no range) damaged my archer who attacked from 3 hexes away. Is that correct?

✅ Found 3 web results:

  1. [PDF] Gloomhaven Rulebook
     URL: https://cdn.1j1ju.com/medias/8d/c5/21-gloomhaven-rulebook.pdf
     Content preview: Example: Archer will focus on the Brute , since she can get in range to attack him with the least nu...

  2. Gloomhaven: Buttons & Bugs Living Rules
     URL: https://cephalofairgames.github.io/bnb-rules/
     Content preview: An attack with no specified range () value is considered a m

0.4375

In [11]:
# Display evaluation metrics
print("\n" + "="*70)
print("EVALUATION METRICS SUMMARY")
print("="*70)
print(f"\nTotal Questions Evaluated: {metrics['total_questions']}")
print(f"\n📊 Accuracy Metrics:")
print(f"  - Is Correct Prediction: {metrics['is_correct_accuracy']:.1%}")
print(f"  - Category Prediction: {metrics['category_accuracy']:.1%}")
print(f"  - Overall Accuracy: {metrics['overall_accuracy']:.1%}")
print("\nNote: Overall accuracy requires both is_correct and category to match.")



EVALUATION METRICS SUMMARY

Total Questions Evaluated: 16

📊 Accuracy Metrics:
  - Is Correct Prediction: 68.8%
  - Category Prediction: 75.0%
  - Overall Accuracy: 43.8%

Note: Overall accuracy requires both is_correct and category to match.


```
User Question
     ↓
LangGraph Agent
     ↓
Retrieve from RAG System (FAISS)
     ↓
Generate Answer with LLM
     ↓
Low Confidence? → Web Search → Enhanced Answer
     ↓
Structured Response (explanation, is_correct, category)
```